In [1]:
import json
from micromort.data_stores.mongodb import getConnection
import carmen
from micromort.data_stores.mysql import db, cursor

In [2]:
mongo_db_name = "micromort"
mongo_collection_name = "newstweets"
mongoClient = getConnection(mongo_db_name, mongo_collection_name)

resolver = carmen.get_resolver()
resolver.load_locations()

/usr/local/lib/python2.7/dist-packages/carmen-0.0.4-py2.7.egg/carmen/resolvers/profile.py:42: UserWarning: Duplicate location name "amman  jordan"
/usr/local/lib/python2.7/dist-packages/carmen-0.0.4-py2.7.egg/carmen/resolvers/profile.py:42: UserWarning: Duplicate location name "amman jordan"
/usr/local/lib/python2.7/dist-packages/carmen-0.0.4-py2.7.egg/carmen/resolvers/profile.py:42: UserWarning: Duplicate location name "st joseph county united states"
/usr/local/lib/python2.7/dist-packages/carmen-0.0.4-py2.7.egg/carmen/resolvers/profile.py:42: UserWarning: Duplicate location name "st tammany parish united states"


In [3]:
def insertInMysql(tweet_location, tweet_id):
    cursor.execute(
                """INSERT INTO newsTweets_location(twitter_url, tweet_id, latitude, longitude, country, state, city, 
                            aliases, resolution_method, known) values(%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
                """,  [tweet_location["twitter_url"], tweet_id, tweet_location["latitude"], 
                       tweet_location["longitude"], tweet_location["country"], tweet_location["state"], 
                       tweet_location["city"], ''.join(tweet_location["aliases"]), tweet_location["resolution_method"], 
                       tweet_location["known"]])

In [4]:
for doc in mongoClient.find():
    try:
        location = resolver.resolve_tweet(doc)
        if location is not None:
            tweet = location[1].__dict__
            tweet["tweet_id"] = doc["id"]
#             tweet["user_location"] = doc["user"]["location"]
#             tweet["place"]  = doc["place"]
#             tweet["coordinates"] = doc["coordinates"]
            if "_id_" in tweet:
                del tweet._id_
            insertInMysql(location[1].__dict__, doc["id"])
    except Exception as ex:
        print ex
        print doc["id"]

/usr/local/lib/python2.7/dist-packages/carmen-0.0.4-py2.7.egg/carmen/resolvers/place.py:49: UserWarning: Tweet has Place with no country
